# Generating Synthetic Dataset from the real one

This is made in order to keep the privacy of the subjects when sharing the dataset

In [1]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns

import plotly.express as px
import plotly.graph_objects as go
import plotly.io as pio

pd.set_option('display.max_columns', None)
# pd.set_option('future.no_silent_downcasting', True)

In [2]:
df_origin = pd.read_parquet('rejection.parquet')
df = df_origin.copy()
df

,recipientAge,recipientWeight,recipientHeight,recipientBodySurface,recipientColor,recipientSex,dialysisType,underlyingDisease,praI,retransplant,isDonorAlive,donorAge,donorSex,donorWeight,donorHeight,donorBmi,donorDeathCause,donorHypertension,donorDiabetesMellitus,donorFinalSodium,donorFinalCpk,donorFinalCreatinine,donorUsedVasoactiveDrug,donorHepatitisCVirus,coldIschemiaTimeMin,totalAnastomosisTimeMin,inductionType,initialImmunization,mismatch,rejected,delayedGraftFunction,outlier_iqr,outlier_zscore,outlier_isolation_forest,outlier_dbscan,null_columns,donorBodySurface,recipientBmi
0,32,87.2,167.0,1.961351,brown,female,hemodialise,hypertension,0.0,False,True,26.0,male,NaN,NaN,NaN,None,False,False,NaN,NaN,1.36,False,None,21.6,35,basiliximab (simulet),CNI+MMF+PRED,6.0,True,False,[recipientBodySurface],[recipientBodySurface],"[mismatch, id]",[id],"[donorWeight, donorHeight, donorBmi, donorDeat...",NaN,31.266808
1,25,46.6,156.0,1.430361,white,female,hemodialise,glomerulopathy,NaN,False,True,29.0,male,NaN,NaN,NaN,None,False,False,NaN,NaN,1.36,False,None,21.6,35,no_induction,CNI+MMF+PRED,3.0,True,False,[],[],[id],[id],"[praI, donorWeight, donorHeight, donorBmi, don...",NaN,19.148586
2,30,49.0,175.0,1.588189,white,female,hemodialise,glomerulopathy,0.0,False,False,50.0,male,75.0,172.0,25.351541,avci,True,False,146.0,181.0,0.87,True,False,22.0,65,basiliximab (simulet),CNI+MMF+PRED,3.0,False,False,[],[],[id],"[donorFinalCpk, id]",[],1.879427,16.000000
3,45,NaN,NaN,NaN,brown,female,hemodialise,glomerulopathy,98.0,True,False,18.0,female,NaN,NaN,NaN,None,False,False,NaN,NaN,1.36,False,None,21.6,48,thymoglobulin,CNI+MMF+PRED,0.0,False,False,"[recipientBodySurface, praI]","[recipientBodySurface, praI]","[praI, id]",[id],"[recipientWeight, recipientHeight, donorWeight...",NaN,NaN
4,55,94.0,176.0,2.103499,white,male,hemodialise,diabetes,NaN,False,True,45.0,female,NaN,NaN,NaN,None,False,False,NaN,NaN,1.36,False,None,21.6,50,no_induction,CNI+MMF+PRED,6.0,True,False,[],[],"[mismatch, id]",[id],"[recipientBirthdate, praI, donorWeight, donorH...",NaN,30.346074
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1250,44,NaN,NaN,NaN,brown,male,None,hypertension,0.0,None,False,19.0,None,NaN,NaN,NaN,tce,False,False,NaN,NaN,1.10,False,None,19.0,48,thymoglobulin,CNI+imTOR+PRED,2.0,False,None,[recipientBodySurface],[recipientBodySurface],[id],[id],"[recipientWeight, recipientHeight, dialysisTyp...",NaN,NaN
1251,34,NaN,NaN,NaN,white,female,None,glomerulopathy,0.0,None,False,35.0,None,NaN,NaN,NaN,tce,False,False,NaN,NaN,1.90,False,None,22.0,48,thymoglobulin,CNI+imTOR+PRED,4.0,False,None,[recipientBodySurface],[recipientBodySurface],[id],[id],"[recipientWeight, recipientHeight, dialysisTyp...",NaN,NaN
1252,59,NaN,NaN,NaN,white,male,None,hypertension,0.0,None,False,35.0,None,NaN,NaN,NaN,avci,False,False,NaN,NaN,1.67,True,None,13.0,48,thymoglobulin,CNI+imTOR+PRED,3.0,False,None,"[recipientBodySurface, coldIschemiaTimeMin]",[recipientBodySurface],[id],[id],"[recipientWeight, recipientHeight, dialysisTyp...",NaN,NaN
1253,35,NaN,NaN,NaN,white,female,None,indeterminate,83.0,None,False,27.0,None,NaN,NaN,NaN,avci,False,False,NaN,NaN,0.80,False,None,11.0,48,None,None,3.0,False,None,"[recipientBodySurface, praI, coldIschemiaTimeMin]",[recipientBodySurface],"[coldIschemiaTimeMin, id]",[id],"[recipientWeight, recipientHeight, dialysisTyp...",NaN,NaN


In [3]:
# all features
num_features =  ['recipientAge', 'recipientWeight', 'recipientHeight', 'recipientBmi', 'recipientBodySurface', 'praI', 'donorAge', 'donorWeight', 'donorHeight', 'donorBmi', 'donorBodySurface', 'donorFinalSodium', 'donorFinalCpk', 'donorFinalCreatinine', 'coldIschemiaTimeMin', 'totalAnastomosisTimeMin', 'mismatch']
cat_features = ['recipientColor', 'recipientSex', 'dialysisType', 'underlyingDisease', 'donorSex', 'donorDeathCause', 'inductionType', 'initialImmunization']
bin_features = ['retransplant', 'donorHepatitisCVirus', 'delayedGraftFunction', 'isDonorAlive', 'donorHypertension', 'donorDiabetesMellitus', 'donorUsedVasoactiveDrug', 'rejected']

df = df[num_features + cat_features + bin_features]
df[bin_features] = df[bin_features].astype('bool')

df_nn = df.dropna(inplace=False)
print(f'shape of df: {df.shape}')
print(f'shape of df_nn: {df_nn.shape}')

print(f'Null row in df_nn: {len(df) - len(df_nn)}')

print(f'df_nn.value_counts(): {df_nn["rejected"].value_counts()}')

shape of df: (1177, 33)
shape of df_nn: (824, 33)
Null row in df_nn: 353
df_nn.value_counts(): rejected
False    740
True      84
Name: count, dtype: int64


/tmp/ipykernel_394046/3183069259.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[bin_features] = df[bin_features].astype('bool')


In [9]:
df_real = df_nn.copy()

# Identify numeric and categorical columns
numeric_cols = df_real.select_dtypes(include=["number"]).columns
categorical_cols = df_real.select_dtypes(exclude=["number"]).columns

# Generate synthetic numeric data by multiplying the real data with a random factor in range [1 - alpha, 1 + alpha]
mean_vals = df_real[numeric_cols].mean()
std_vals = df_real[numeric_cols].std()

# Generate scaling factors using mean and std
demographic_alpha = 0.4
clinical_alpha = 0.2

demographic_numeric_cols = ['recipientAge', 'recipientWeight', 'recipientHeight', 'recipientBodySurface', 'donorAge', 'donorWeight', 'donorHeight', 'donorBmi']
clinical_numeric_cols = ['praI', 'donorFinalSodium', 'donorFinalCpk', 'donorFinalCreatinine', 'coldIschemiaTimeMin', 'totalAnastomosisTimeMin', 'mismatch']

random_demographic_factors = np.random.uniform(1 - demographic_alpha * (std_vals[demographic_numeric_cols] / mean_vals[demographic_numeric_cols]), 1 + demographic_alpha * (std_vals[demographic_numeric_cols] / mean_vals[demographic_numeric_cols]), size=df_real[demographic_numeric_cols].shape)
random_clinical_factors = np.random.uniform(1 - clinical_alpha * (std_vals[clinical_numeric_cols] / mean_vals[clinical_numeric_cols]), 1 + clinical_alpha * (std_vals[clinical_numeric_cols] / mean_vals[clinical_numeric_cols]), size=df_real[clinical_numeric_cols].shape)

df_synthetic = df_real.copy()
df_synthetic[demographic_numeric_cols] = df_real[demographic_numeric_cols] * random_demographic_factors
df_synthetic[clinical_numeric_cols] = df_real[clinical_numeric_cols] * random_clinical_factors

int_cols = ['recipientAge', 'recipientWeight', 'recipientHeight', 'praI', 'donorAge', 'donorWeight', 'donorHeight', 'totalAnastomosisTimeMin', 'mismatch']
df_synthetic[int_cols] = df_synthetic[int_cols].astype(int)

df_synthetic['recipientBodySurface'] = df_synthetic['recipientWeight'] ** 0.425 * df_synthetic['recipientHeight'] ** 0.725 * 0.007184
df_synthetic['donorBodySurface'] = df_synthetic['donorWeight'] ** 0.425 * df_synthetic['donorHeight'] ** 0.725 * 0.007184
df_synthetic['recipientBmi'] = df_synthetic['recipientWeight'] / (df_synthetic['recipientHeight'] / 100) ** 2
df_synthetic['donorBmi'] = df_synthetic['donorWeight'] / (df_synthetic['donorHeight'] / 100) ** 2

# Keep categorical values unchanged
df_synthetic[categorical_cols] = df_real[categorical_cols]
df_synthetic['praI'] = df_real['praI']
df_synthetic['mismatch'] = df_real['mismatch']

# Save or return the synthetic dataset
df_synthetic.to_csv("synthetic_dataset.csv", index=False)


In [10]:
from scipy.stats import ks_2samp

# Initialize a list to store the results
ks_results = []

# Perform the KS test for each column and store the results, only for numeric columns
for col in numeric_cols:
    stat, p_value = ks_2samp(df_real[col], df_synthetic[col])
    ks_results.append({'Column': col, 'KS Statistic': stat, 'p-value': p_value})

# Convert the results to a DataFrame
ks_results_df = pd.DataFrame(ks_results)
ks_results_df


,Column,KS Statistic,p-value
0,recipientAge,0.043689,0.411421
1,recipientWeight,0.040049,0.523573
2,recipientHeight,0.042476,0.447256
3,recipientBmi,0.031553,0.806945
4,recipientBodySurface,0.041262,0.484701
5,praI,0.000000,1.000000
6,donorAge,0.046117,0.345063
7,donorWeight,0.083738,0.006170
8,donorHeight,0.077670,0.013847
9,donorBmi,0.059466,0.108523


In [11]:
display(df_real.head())
display(df_synthetic.head())
print(f'df_real.shape: {df_real.shape}, df_synthetic.shape: {df_synthetic.shape}')

,recipientAge,recipientWeight,recipientHeight,recipientBmi,recipientBodySurface,praI,donorAge,donorWeight,donorHeight,donorBmi,donorBodySurface,donorFinalSodium,donorFinalCpk,donorFinalCreatinine,coldIschemiaTimeMin,totalAnastomosisTimeMin,mismatch,recipientColor,recipientSex,dialysisType,underlyingDisease,donorSex,donorDeathCause,inductionType,initialImmunization,retransplant,donorHepatitisCVirus,delayedGraftFunction,isDonorAlive,donorHypertension,donorDiabetesMellitus,donorUsedVasoactiveDrug,rejected
2,30,49.0,175.0,16.000000,1.588189,0.0,50.0,75.0,172.0,25.351541,1.879427,146.0,181.0,0.87,22.00,65,3.0,white,female,hemodialise,glomerulopathy,male,avci,basiliximab (simulet),CNI+MMF+PRED,False,False,False,False,True,False,True,False
5,26,60.0,177.0,19.151585,1.745264,0.0,22.0,55.0,160.0,21.484375,1.563174,135.0,35.0,0.40,32.08,45,4.0,brown,male,hemodialise,hypertension,female,tce,basiliximab (simulet),CNI+MMF+PRED,False,False,True,False,False,False,True,False
6,40,43.5,143.0,21.272434,1.304196,0.0,10.0,22.0,130.0,13.017751,0.910972,162.0,34742.0,1.50,28.33,40,4.0,brown,female,hemodialise,glomerulopathy,female,others,basiliximab (simulet),CNI+MMF+PRED,False,False,True,False,False,False,True,True
7,66,70.0,158.0,28.040378,1.716166,0.0,25.0,70.0,180.0,21.604938,1.886277,142.0,1262.0,1.10,21.92,55,1.0,brown,female,dialise_peritoneal,indeterminate,male,tce,basiliximab (simulet),CNI+AZA+PRED,False,False,False,False,False,False,False,False
8,34,68.0,152.0,29.432133,1.648235,0.0,25.0,70.0,180.0,21.604938,1.886277,142.0,1262.0,1.10,26.00,85,1.0,white,female,hemodialise,glomerulopathy,male,tce,basiliximab (simulet),CNI+MMF+PRED,False,False,False,False,False,False,False,False


,recipientAge,recipientWeight,recipientHeight,recipientBmi,recipientBodySurface,praI,donorAge,donorWeight,donorHeight,donorBmi,donorBodySurface,donorFinalSodium,donorFinalCpk,donorFinalCreatinine,coldIschemiaTimeMin,totalAnastomosisTimeMin,mismatch,recipientColor,recipientSex,dialysisType,underlyingDisease,donorSex,donorDeathCause,inductionType,initialImmunization,retransplant,donorHepatitisCVirus,delayedGraftFunction,isDonorAlive,donorHypertension,donorDiabetesMellitus,donorUsedVasoactiveDrug,rejected
2,26,52,170,17.993080,1.594936,0.0,53,75,172,25.351541,1.879427,144.123752,146.790156,0.893670,21.851294,64,3.0,white,female,hemodialise,glomerulopathy,male,avci,basiliximab (simulet),CNI+MMF+PRED,False,False,False,False,True,False,True,False
5,27,56,177,17.874812,1.694833,0.0,22,55,163,20.700817,1.584370,136.609047,17.928761,0.421254,31.762669,46,4.0,brown,male,hemodialise,hypertension,female,tce,basiliximab (simulet),CNI+MMF+PRED,False,False,True,False,False,False,True,False
6,39,44,143,21.516945,1.310546,0.0,10,23,133,13.002431,0.943829,164.229452,26096.353702,1.530983,28.817619,41,4.0,brown,female,hemodialise,glomerulopathy,female,others,basiliximab (simulet),CNI+MMF+PRED,False,False,True,False,False,False,True,True
7,63,74,159,29.270994,1.765236,0.0,22,72,180,22.222222,1.908997,142.168711,1547.052629,1.207032,21.573128,54,1.0,brown,female,dialise_peritoneal,indeterminate,male,tce,basiliximab (simulet),CNI+AZA+PRED,False,False,False,False,False,False,False,False
8,35,71,154,29.937595,1.694742,0.0,27,74,183,22.096808,1.954640,141.980193,1578.878859,0.971920,26.383388,81,1.0,white,female,hemodialise,glomerulopathy,male,tce,basiliximab (simulet),CNI+MMF+PRED,False,False,False,False,False,False,False,False


df_real.shape: (824, 33), df_synthetic.shape: (824, 33)


In [12]:
# plot ks_results_df
fig = px.bar(ks_results_df, x='Column', y='KS Statistic', title='KS Statistic for each column')
fig.show()

fig = px.bar(ks_results_df, x='Column', y='p-value', title='p-value for each column')
fig.show()

In [13]:
# plot histogram of donorWeight, in order to check the distribution of donorWeight in real and synthetic data
fig = px.histogram(df_real, x='donorWeight', title='donorWeight distribution in real data', range_x=[0, 150])
fig.show()
fig = px.histogram(df_synthetic, x='donorWeight', title='donorWeight distribution in real data', range_x=[0, 150])
fig.show()

In [14]:
df_synthetic.to_parquet('synthetic_dataset.parquet')